For EDA part

- You need to analyze the data and display at least 5 different insights in the dashboard.
- For each insight drawn from the data, you need to create charts and add a description for it.

[10 points] EDA 
    
- provide 5 insights (2 points for each)

## 1. Non-interactive analysis via spark-submit

submitting ```model.py```.

NOTE:  You should add some jars to properly import the HIVE tables using Spark SQL.

In [ ]:
spark-submit --jars /usr/hdp/current/hive-client/lib/hive-metastore-1.2.1000.2.6.5.0-292.jar,/usr/hdp/current/hive-client/lib/hive-exec-1.2.1000.2.6.5.0-292.jar --packages org.apache.spark:spark-avro_2.12:3.0.3 scripts/model.py 

1. Connect to Hive

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .appName("BDT Project")\
        .config("spark.sql.catalogImplementation","hive")\
        .config("hive.metastore.uris", "thrift://sandbox-hdp.hortonworks.com:9083")\
        .config("spark.sql.avro.compression.codec", "snappy")\
        .enableHiveSupport()\
        .getOrCreate()

2. List all databases

In [ ]:
print(spark.catalog.listDatabases())

3. List all tables

TODO: specify table name

In [ ]:
TABLE_NAME = '?'

In [ ]:
print(spark.catalog.listTables(TABLE_NAME))

4. Read Hive table

In [ ]:
emps = spark.read.format("avro").table(f'{TABLE_NAME}.employees_part')
emps.createOrReplaceTempView('employees')

depts = spark.read.format("avro").table(f'{TABLE_NAME}.departments_buck')
depts.createOrReplaceTempView('departments')

5. Run some queries

In [ ]:
emps.printSchema()
depts.printSchema()

spark.sql("SELECT * FROM employees WHERE deptno=10").show()

spark.sql("SELECT * FROM departments").show()

## 2. Interactive analysis via Zeppelin

1. Create a new note and set python2 as the default interpter.

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline


spark = SparkSession.builder\
    .appName("BDT Project")\
    .master("local[*]")\
    .config("hive.metastore.uris", "thrift://sandbox-hdp.hortonworks.com:9083")\
    .config("spark.sql.catalogImplementation","hive")\
    .config("spark.sql.avro.compression.codec", "snappy")\
    .config("spark.jars", "file:///usr/hdp/current/hive-client/lib/hive-metastore-1.2.1000.2.6.5.0-292.jar,file:///usr/hdp/current/hive-client/lib/hive-exec-1.2.1000.2.6.5.0-292.jar")\
    .config("spark.jars.packages","org.apache.spark:spark-avro_2.12:3.0.3")\
    .enableHiveSupport()\
    .getOrCreate()


sc = spark.sparkContext

print(sc)

3. List all databases

In [ ]:
print(spark.catalog.listDatabases())

4. Read Hive table

In [ ]:
emps = spark.read.format("avro").table('projectdb.employees_part')
emps.createOrReplaceTempView('employees')

depts = spark.read.format("avro").table('projectdb.departments_buck')
emps.createOrReplaceTempView('departments')

5. Run some queries

In [ ]:
emps.printSchema()
depts.printSchema()

spark.sql("SELECT * FROM employees WHERE deptno=10").show()

spark.sql("SELECT * FROM departments").show()

spark.sql("SELECT AVG(SAL) FROM employees;").show()
spark.sql("SELECT * from employees where comm is NULL;").show()

---

## Exploring features

We will use pyspark for interactive analysis

In [ ]:
# TODO: take the code from above

In [ ]:
df = spark.read.format("avro").table("my_database.trips")

### Insight 1 - 

In [ ]:
# get the shape of the DataFrame
print((df.count(), len(df.columns)))

# get the column names of the DataFrame
print(df.columns)

# show the first 10 rows of the DataFrame
df.show(10)

In [ ]:
# filter out categorical features

cat_cols = [field.name for field in df.schema.fields if isinstance(
    field.dataType, StringType) or isinstance(field.dataType, LongType)]

print("Result: ", cat_cols)

In [ ]:
# get the number of missing values in each column of the DataFrame
missing_vals = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

# display the results
missing_vals.show()

**Insight 1**: "There are many missing values in ORIGIN_CALL and ORIGIN_STAND because may be all the taxi users have not called the via phone and they have not started their trip from taxi stand"

### Insight 2 - 

In [ ]:
# display the schema of the DataFrame
df.printSchema()

# compute summary statistics for all numeric columns
df.describe().show()

In [ ]:
from pyspark.sql.functions import col

# compute summary statistics for all categorical columns
df.select([col(c).describe() for c in df.columns if c in cat_cols]).show()

**Insight 2**: "We can see the DAY_TYPE has only 1 unique value and that is 'A' which means that all the trips are started on normal day or weekend. 

Also the 5901 observations don't have the POLYLINE values means we cannot calculate the travel time for those trips."

### Insight 3 - 

In [ ]:
from pyspark.sql.functions import asc

# sort the DataFrame by the timestamp column in ascending order
df_sorted = df.orderBy('timestamp_col', ascending=True)

In [ ]:
from pyspark.sql.functions import date_format, to_date, dayofweek

# add new columns for year, month, day, hour, and day of the week
df = df.withColumn('year', date_format('timestamp_col', 'y')) \
    .withColumn('month', date_format('timestamp_col', 'M')) \
    .withColumn('day', date_format('timestamp_col', 'd')) \
    .withColumn('hour', date_format('timestamp_col', 'H')) \
    .withColumn('day_of_week', dayofweek(to_date('timestamp_col'))) 

In [ ]:
# Pie chart for the year

# TODO: rewrite this for zeppelin
plt.figure(figsize = (10,10))
plt.pie(df['year'].value_counts(), labels = df['year'].value_counts().keys(),autopct = '%.1f%%')


**Insight 3**: "From the above pie chart it is clear that there are equal number of taxi trips in both the year."

### Insight 4 - 

In [ ]:
# Chart of trips per day of week

# TODO: rewrite this for zeppelin
plt.figure(figsize = (5,5))
plt.title('Count of trips per day of week')
sns.countplot(y = 'week_day', data = df)
plt.xlabel('Count')
plt.ylabel('Day')

**Insight 4**: "The 4th and 5th day of week has almost same number of trips and rest all the days have almost similar number of trips. This means that we can say each and every day of week required same number of taxies irrespective of weekend or working day."

### Insight 5 - 

In [ ]:
# chart of trips per month

# TODO: rewrite this for zeppelin

plt.figure(figsize = (10,10))
plt.title('Count of trips per month')
sns.countplot(y = 'month', data = df)
plt.xlabel('Count')
plt.ylabel('Month')

**Insight 5**: "On an average we can say that every month has atleast 120000 taxi trips planned."

### Insight 6

In [ ]:
# chart of trips per hour

# TODO: rewrite this for zeppelin


plt.figure(figsize = (10,10))
plt.title('Count of trips per hour')
sns.countplot(y = 'hour', data = df)
plt.xlabel('Count')
plt.ylabel('Hours')




**Insight 6**: "14th and 15th hour may be the peak hours, office time, school time because lot of taxies are used between this time."

### Insight 7

drop null polylines

convert polyline into total travel time

**Insight 7**: "Above description clear that the minimum travelling time by the taxi is 15 seconds and maximum is 58200 seconds i.e.around 16 hours 16 min."
